In [ ]:
# from metavision_core.event_io import EventsIterator, load_events
import tonic
import matplotlib.pyplot as plt
import numpy as np
import torchvision
import torch.nn as nn
import torch
import models
import random
from IPython.utils import io
# import sdl2.ext


In [ ]:
# %matplotlib inline
%matplotlib ipympl
# %matplotlib tk
plt.rcParams["image.cmap"] = "bone"

In [ ]:
file_name = "1hzplane"

# ievs = load_events("./data/Line/" + file_name + ".dat")
# ievs = np.load("./data/Line/" + file_name + ".npy")
# ifs = torch.from_numpy(
#     np.load("./data/Line/"+file_name+"_frames.npy")[0:2000,:,:]
#     )
# if torch.cuda.is_available():
#   ifs = ifs.cuda(0)

In [ ]:
in_data = tonic.datasets.NMNIST(save_to="./data") # (34, 34, 2) 3
# i_evs = tonic.datasets.POKERDVS(save_to="./data") # (35, 35, 2) 3
# i_evs = tonic.datasets.DVSGesture(save_to="./data") # (128, 128, 2) 2
# i_evs = tonic.datasets.ASLDVS(save_to="./data") # (240, 180, 2) 3

in_data.sensor_size

In [ ]:
noise = tonic.transforms.UniformNoise(
    sensor_size=in_data.sensor_size[:-1]+(1,),
    n=int(4172),
)
to_frame = tonic.transforms.ToFrame(in_data.sensor_size[:-1]+(1,), n_time_bins=100)

In [ ]:
events, label = in_data[0]#[random.randint(0, len(in_data)-1)]
events = events.squeeze()
events = np.array(list(filter(lambda ev: ev[3]==0, events)))
# print(events)
print(label)

true_frames = to_frame(events)
in_frames = to_frame(noise(events))

In [ ]:
# ani.save("./img/nmnist-9.gif", writer="imagemagick")

In [ ]:
# def n_evs(evs):
#     sum = 0
#     for i, ev in enumerate(evs):
#         # print(i)
#         sum = sum + (ev[0].size-sum)/(i+1)
#     print(sum)
# n_evs(i_evs)

# # MNIST: 4172
# # POKERDVS: 2991
# # DVSGesture: 361903
# # ASLDVS: 28149
# # CIFAR10DVS: 204409

In [ ]:
# def t_evs(evs):
#     sum = 0
#     for i, ev in enumerate(evs):
#         # print(i)
#         sum = sum + (ev[0][-1][0]-sum)/(i+1)
#     print(sum)
# t_evs(i_evs)

# # MNIST: 307663
# # POKERDVS: 17283
# # DVSGesture: 6455240
# # ASLDVS: 110247
# # CIFAR10DVS: 1293438

In [ ]:
in_frames.shape

In [ ]:
net = models.Blur1()
in_frames = torch.tensor(in_frames).float()
with torch.inference_mode():
    out_frames = net(in_frames)

In [ ]:
plt.rc('animation', html='jshtml')
with io.capture_output() as captured:
    anim = tonic.utils.plot_animation(np.concatenate((in_frames,out_frames,true_frames),axis=3))
display(anim)